### 분류 모델 수행하기
고객 891명에 대한 학습용 데이터를 이용하여 생존 여부를 예측하는 모형을 만듭니다.<br>
이후 예측 모형을 평가용 데이터에 적용하여 418명 승객의 생존 여부 예측값을 다음과 같은 형식의 CSV 파일로 생성하시오<br>
(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)

### 2.1 데이터 탐색하기 

데이터 가져오기

In [1]:
import pandas as pd

In [2]:
x_train = pd.read_csv('titanic_x_train.csv', encoding='CP949')
x_test = pd.read_csv('titanic_x_test.csv', encoding='CP949')
y_train = pd.read_csv('titanic_y_train.csv')

In [3]:
# x_train, x_test 상위 1개 행 확인하기
print(x_train.head(1).T)

                                   0
PassengerId                        1
티켓등급                               3
승객이름         Braund, Mr. Owen Harris
성별                              male
나이                              22.0
형제자매배우자수                           1
부모자식수                              0
티켓번호                       A/5 21171
운임요금                            7.25
객실번호                             NaN
선착장                                S


In [4]:
print(x_test.head(1).T)

                            0
PassengerId               892
티켓등급                        3
승객이름         Kelly, Mr. James
성별                       male
나이                       34.5
형제자매배우자수                    0
부모자식수                       0
티켓번호                   330911
운임요금                   7.8292
객실번호                      NaN
선착장                         Q


In [5]:
print(y_train.head(5))

   PassengerId  Survived
0            1         0
1            2         1
2            3         1
3            4         1
4            5         0


행/열 확인하기

In [6]:
# 각 데이터 세트의 행과 열의 개수를 확인하기
print(x_train.shape) # 11개 칼럼과 891건의 데이터 세트

(891, 11)


In [7]:
print(x_test.shape) #11개 칼럼과 418건의 데이터 세트

(418, 11)


In [8]:
print(y_train.shape) # 2개 칼럼과 891건의 데이터 세트

(891, 2)


요약정보 확인하기

In [9]:
# x_train 세트의 요약정보 확인하기
print(x_train.info()) #승객이름 , 성별, 티켓번호, 객실번호, 선착장은 문자형변수

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   티켓등급         891 non-null    int64  
 2   승객이름         891 non-null    object 
 3   성별           891 non-null    object 
 4   나이           714 non-null    float64
 5   형제자매배우자수     891 non-null    int64  
 6   부모자식수        891 non-null    int64  
 7   티켓번호         891 non-null    object 
 8   운임요금         891 non-null    float64
 9   객실번호         204 non-null    object 
 10  선착장          889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB
None


In [10]:
# 성별 칼럼의 중복 제거한 값과 개수 확인하기
print(x_train['성별'].unique().size)

2


In [11]:
print(x_train['성별'].unique())

['male' 'female']


In [12]:
#선착장 칼럼의 중복 제거한 값과 개수 확인하기
print(x_train['선착장'].unique().size)

4


In [13]:
print(x_train['선착장'].unique()) # 3개의 값과 결측치

['S' 'C' 'Q' nan]


In [14]:
# 나머지 3개의 칼럼의 중복제거한 값의 개수 확인하기
print(x_train['승객이름'].unique().size)
print(x_train['티켓번호'].unique().size)
print(x_train['객실번호'].unique().size)
# 소수의 값인 성별과 선착장은 범주형 데이터 나머지는 문자열값

891
681
148


기초 통계량 확인하기

In [15]:
# x_train의 기초 통계량을 확인하고 가독성을 위해 행/열 바꿔서 출력하기
print(x_train.describe().T)

             count        mean         std   min       25%       50%    75%  \
PassengerId  891.0  446.000000  257.353842  1.00  223.5000  446.0000  668.5   
티켓등급         891.0    2.308642    0.836071  1.00    2.0000    3.0000    3.0   
나이           714.0   29.699118   14.526497  0.42   20.1250   28.0000   38.0   
형제자매배우자수     891.0    0.523008    1.102743  0.00    0.0000    0.0000    1.0   
부모자식수        891.0    0.381594    0.806057  0.00    0.0000    0.0000    0.0   
운임요금         891.0   32.204208   49.693429  0.00    7.9104   14.4542   31.0   

                  max  
PassengerId  891.0000  
티켓등급           3.0000  
나이            80.0000  
형제자매배우자수       8.0000  
부모자식수          6.0000  
운임요금         512.3292  


독립변수와 종속변수의 관계 확인하기

In [16]:
# x_train, y_train을 세로 방향으로 통합한 후 data 변수에 저장하기
data=pd.concat([x_train,y_train],axis=1)

In [17]:
# 성별 칼럼에 따라 Survived의 평균값을 구하기
print(data.groupby(['성별'])['Survived'].mean()) # 여성이 남성보다 생존율이 높음을 알 수 있음

성별
female    0.742038
male      0.188908
Name: Survived, dtype: float64


In [18]:
# 티켓등급 칼럼에 따라 Survived의 평균값을 구하기
print(data.groupby(['티켓등급'])['Survived'].mean()) #티켓등급이 높을 수록 생존율이 높음

티켓등급
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64


In [19]:
# 선착장 칼럼에 따라 Survived의 평균값 구하기
print(data.groupby(['선착장'])['Survived'].mean()) # C 선착장이 가장 생존율 가장 높음

선착장
C    0.553571
Q    0.389610
S    0.336957
Name: Survived, dtype: float64


이러한 결과를 통해 성별 티켓등급 선착장 칼럼들은 최종적으로 분류해야하는 Survived 칼럼과 명확한 관계가 있으므로 포함해야함

### 2.2 전처리하기 

불필요한 칼럼 삭제

In [20]:
# 테스트 데이터의 PassengerId 값은 x_test_passenger_id 변수에 저장하기
x_test_passenger_id = x_test['PassengerId']

In [21]:
#PassengerId 칼럼을 삭제하기
x_train= x_train.drop(columns=['PassengerId'])
x_test= x_test.drop(columns=['PassengerId'])
y_train= y_train.drop(columns=['PassengerId'])

In [22]:
# 칼럼이 삭제된 상위 5개 행을 확인하기
print(x_train.head(5))

   티켓등급                                               승객이름      성별    나이  \
0     3                            Braund, Mr. Owen Harris    male  22.0   
1     1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
2     3                             Heikkinen, Miss. Laina  female  26.0   
3     1       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
4     3                           Allen, Mr. William Henry    male  35.0   

   형제자매배우자수  부모자식수              티켓번호     운임요금  객실번호 선착장  
0         1      0         A/5 21171   7.2500   NaN   S  
1         1      0          PC 17599  71.2833   C85   C  
2         0      0  STON/O2. 3101282   7.9250   NaN   S  
3         1      0            113803  53.1000  C123   S  
4         0      0            373450   8.0500   NaN   S  


In [23]:
print(y_train.head())

   Survived
0         0
1         1
2         1
3         1
4         0


In [24]:
#학습 데이터와 테스트 데이터에서 '승객이름' 칼럼 삭제하기
x_train = x_train.drop(columns=['티켓번호'])
x_test = x_test.drop(columns=['티켓번호'])

In [25]:
#학습 데이터와 테스트 데이터에서 '승객이름' 칼럼 삭제하기
x_train = x_train.drop(columns=['승객이름'])
x_test = x_test.drop(columns=['승객이름'])

결측치 처리하기

In [26]:
# 나이, 객실번호 선착장 칼럼의 결측치 개수 세기
print(x_train['나이'].isnull().sum())
print(x_train['객실번호'].isnull().sum())
print(x_train['선착장'].isnull().sum())

177
687
2


In [27]:
# 나이와 Survived 칼럼간의 상관관계 구하기
print(data[['나이','Survived']].corr()) # 상관관계가 매우낮아 분석대상에서 삭제

                나이  Survived
나이        1.000000 -0.077221
Survived -0.077221  1.000000


In [28]:
#x_train에서 나이 칼럼을 삭제하기
x_train=x_train.drop(columns=['나이'])

In [29]:
#x_test에서 나이 칼럼 삭제
x_test = x_test.drop(columns=['나이'])

In [30]:
#x_train의 객실번호 값에서 중복을 제외한 값의 개수 세기
print(x_train['객실번호'].unique().size) # 거의 서로 상이한 값이라 결측치 처리가 어려워 분석대상에 제외

148


In [31]:
# x_train에서 객실번호 칼럼 삭제
x_train=x_train.drop(columns=['객실번호'])

In [32]:
# x_test에서 객실번호 삭제
x_test = x_test.drop(columns=['객실번호'])

In [33]:
# 선착장칼럼은 학습데이터의 결측치가 2건이므로 최빈값으로 대체 선착장 칼럼별로 값의 개수를 세기
print(x_train.groupby(['선착장'])['선착장'].count()) # 결측치는 가장많은 S로 대체

선착장
C    168
Q     77
S    644
Name: 선착장, dtype: int64


In [34]:
# x_train의 선착장 칼럼 결측치는 'S' 로 적용
x_train['선착장'] = x_train['선착장'].fillna('S')

In [35]:
# x_train 선착장 칼럼에 결측치가 있는지 다시확인
print(x_train['선착장'].isnull().sum())

0


In [36]:
#x_test 선착장 칼럼에 결측치 다시 확인
print(x_test['선착장'].isnull().sum())

0


범주형 변수를 인코딩하기

In [37]:
# 학습 데이터의 성별 칼럼을 인코딩하기
x_train['성별']=x_train['성별'].replace('male',0).replace('female',1)

In [38]:
# 테스트 데이터의 성별 칼럼을 인코딩하기
x_test['성별']=x_test['성별'].replace('male',0).replace('female',1)

In [39]:
#x_train의 선착장 칼럼에 대해 원핫 인코딩을 수행한 후 선착장_dummy에 저장하기
선착장_dummy = pd.get_dummies(x_train['선착장'], drop_first = True).rename(columns={'Q':'선착장Q','S':'선착장S'})

In [40]:
#기존 x_train의 우측에 '선착장_dummy' 변수를 덧붙여 x_train에 다시 저장하기
x_train = pd.concat([x_train, 선착장_dummy],axis=1)

In [41]:
#x_train의 상위 5개 행 확인
print(x_train.head())

   티켓등급  성별  형제자매배우자수  부모자식수     운임요금 선착장  선착장Q  선착장S
0     3   0         1      0   7.2500   S     0     1
1     1   1         1      0  71.2833   C     0     0
2     3   1         0      0   7.9250   S     0     1
3     1   1         1      0  53.1000   S     0     1
4     3   0         0      0   8.0500   S     0     1


In [42]:
# 학습 데이터에서 선착장 칼럼 삭제하기
x_train = x_train.drop(columns =['선착장'])

In [43]:
#x_test 세트의 선착장 칼럼에 대해 원핫 인코딩을 수행하고 기존 칼럼은 삭제하기
선착장_dummy=pd.get_dummies(x_test['선착장'], drop_first = True).rename(columns={'Q':'선착장Q','S':'선착장S'})

In [44]:
# 기존 x_test의 우측에 선착장_dummy 변수를 덧부여 x_test에 다시 저장하기
x_test = pd.concat([x_test,선착장_dummy],axis=1)

In [45]:
#테스트 데이터에서 선착장 칼럼 삭제하기
x_test= x_test.drop(columns=['선착장'])

파생변수 만들기<br>
데이터 세트에는 형제자매배우자수 칼럼과 부모자식수 칼럼이 있습니다.<br>
두 칼럼 모두 가족이란 의미를 지닌 유사 칼럼이므로 각 칼럼을 더해 가족수라는 파생변수 만들기

In [46]:
# 형제자매배우자수와 부모자식수 칼럼을 더하여 가족수라는 칼럼만들기
x_train['가족수']=x_train['형제자매배우자수']+x_train['부모자식수']

In [47]:
#결과 확인을 위해 상위 10개 행을 확인하기
print(x_train[['형제자매배우자수','부모자식수','가족수']].head(10))

   형제자매배우자수  부모자식수  가족수
0         1      0    1
1         1      0    1
2         0      0    0
3         1      0    1
4         0      0    0
5         0      0    0
6         0      0    0
7         3      1    4
8         0      2    2
9         1      0    1


In [48]:
# 학습데이터에서 형제자매배우자수, 부모자식수 칼럼 삭제하기
x_train = x_train.drop(columns=['형제자매배우자수','부모자식수'])

In [49]:
# 테스트 데이터의 형제자매배우자수와 부모자식수 칼럼을 더한 가족수라는 칼럼 만들기
x_test['가족수']=x_test['형제자매배우자수']+x_test['부모자식수']
# 테스트데이터에서 형제자매배우자수, 부모자식수 칼럼 삭제하기
x_test = x_test.drop(columns=['형제자매배우자수','부모자식수'])

### 2.3 학습하고 평가하기
1. 모델 학습시키기 2.평가지표계산하기 3. y_test 예측값 or 예측확률 계산하기

데이터 분리하기

In [50]:
# 데이터 분리하기 위한 train_test_split 함수를 가져오기
from sklearn.model_selection import train_test_split

In [51]:
# 학습용과 검증용을 8:2로 분리하여 각 4개의 변수에 저장하기
X_TRAIN,X_TEST,Y_TRAIN,Y_TEST = train_test_split(x_train,y_train,\
                                                test_size=0.2, random_state=10)

In [52]:
#분리된 데이터의 행/열 구조 파악
print(X_TRAIN.shape)
print(X_TEST.shape)
print(Y_TRAIN.shape)
print(Y_TEST.shape)

(712, 6)
(179, 6)
(712, 1)
(179, 1)


데이터 학습 및 하이퍼 파라미터 튜닝하기<br>
학습할 준비가 완료되면 종속변수인 Survived(생존여부)값을 분류하는 모델을 만들기<br>
이번 분류모델에서는 일반적으로 성능이 잘나오는 XGB 분류기를 활용하여 학습 모델을 생성

In [53]:
#xgboost 라이브러리에서 제공하는 XGBClassifier 함수 가져오기
from xgboost import XGBClassifier

In [63]:
# XGB 분류 분석을 수행할 첫 번째 모델을 만들고 공부시키기
model = XGBClassifier(eval_metric = 'error', random_state=10)
model.fit(X_TRAIN, Y_TRAIN)

C:\Users\Hello Beautiful\Desktop\anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Hello Beautiful\Desktop\anaconda\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Hello Beautiful\Desktop\anaconda\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using 

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='error', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, predictor='auto', random_state=10,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [70]:
# XGB 분류 분석을 수행할 두 번째 모델을 만들고 공부시키기
model = XGBClassifier(n_estimators =100 , max_depth = 5, \
                      eval_metric = 'error', random_state=10)
#만들어진 모델을 공부시키기
model.fit(X_TRAIN, Y_TRAIN)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='error', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, predictor='auto', random_state=10,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

결과 예측하기<br>
종속변수인 Survived(생존 여부)값을 예측하기 위해서 생성한 모델에 predict() 함수를 사용하여 예측<br>
predict()함수로 추출된 결과는 데이터 프레임 형태가 아니므로 반드시 pd.DataFrame()함수를 사용해 데이터 타입을 변경하기

In [71]:
# 학습이 완료된 모델을 통해 y_test 예측하기 : 최종 결과 제출용
y_test_predicted = pd.DataFrame(model.predict(x_test)).rename(columns={0:'Survived'})

In [72]:
# y_test_predicted 값 확인하기
print(pd.DataFrame(y_test_predicted.head(10)))

   Survived
0         0
1         1
2         0
3         0
4         1
5         0
6         0
7         1
8         1
9         0


만약 승객이 사망할 확률을 구해야한다면 pd.DataFrame(model.predict_proba(x_test))[0] 코드, 생존해야한다면 [1] 코드를 작성<br>
향후 ROC 평가지표를 계산하기 위해 X_TEST 검증용 세트에 대응되는 Y_TEST 값을 예측함

In [73]:
# 학습이 완료된 모델을 통해 Y_TEST 값을 예측하기 : 평가지표 계산용
Y_TEST_PREDICTED = pd.DataFrame(model.predict(X_TEST))

모델 평가하기

In [67]:
# sklearn 패키지의 metrics 모듈에서 roc_auc_score 함수 가져오기
from sklearn.metrics import roc_auc_score

In [68]:
# 1차 학습 모델의 ROC 평가지표 값을 확인하기
print(roc_auc_score(Y_TEST,Y_TEST_PREDICTED))

0.7731596360628619


In [74]:
# 2차 학습 모델의 ROC 평가지표 값을 확인하기
print(roc_auc_score(Y_TEST,Y_TEST_PREDICTED))
#최종 제출은 2번째로 선택

0.7983181692859112


### 2.4 결과 제출하기
최종 데이터는 PassengerId 변수와 y_test 세트의 예측값(y_test_predicted)을 함께 출력해야함<br>
따라서 concat() 함수를 사용하여 이전에 저장한 x_test_passenger_id 변수와 y_test_predicted 변수를 세로로 통합

In [75]:
print(pd.concat([x_test_passenger_id,y_test_predicted],axis=1))

     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         1

[418 rows x 2 columns]


In [76]:
# 앞의 출력결과를 final 변수에 저장하기
final = pd.concat([x_test_passenger_id,y_test_predicted],axis=1)

In [77]:
# final 변수를 123456.csv 이름으로 저장하기
final.to_csv('123456.csv',index=False)

In [80]:
# 제출확인하기
final = pd.read_csv('123456.csv')
print(final)

     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         1

[418 rows x 2 columns]
